In [1]:
import numpy as np
import pandas as pd

In [2]:
books = pd.read_csv('books.csv')
ratings = pd.read_csv('ratings.csv')
users = pd.read_csv('users.csv')

c:\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
print(books.shape)
print(ratings.shape)
print(users.shape)

(271360, 8)
(1149780, 3)
(278858, 3)


In [4]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [5]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [6]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [7]:
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [8]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [9]:
print(list(books.columns))

['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']


In [10]:
print(list(ratings.columns))

['User-ID', 'ISBN', 'Book-Rating']


In [11]:
print(list(users.columns))

['User-ID', 'Location', 'Age']


In [12]:
ratings_with_name = books.merge(ratings, on='ISBN')

In [13]:
ratings_with_name.shape

(1031136, 10)

In [14]:
num_rating_df = ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_df.rename(columns = {'Book-Rating':'num-rating'}, inplace=True)
num_rating_df

,Book-Title,num-rating
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [15]:
avg_rating_df = ratings_with_name.groupby('Book-Title').mean()['Book-Rating'].reset_index()
avg_rating_df.rename(columns = {'Book-Rating':'avg-rating'}, inplace=True)
avg_rating_df

,Book-Title,avg-rating
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241066,Ã?Â?lpiraten.,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,5.250000
241068,Ã?Â?sterlich leben.,7.000000
241069,Ã?Â?stlich der Berge.,2.666667


In [16]:
popular_df = num_rating_df.merge(avg_rating_df, on='Book-Title')
popular_df

,Book-Title,num-rating,avg-rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,2,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Â?sterlich leben.,1,7.000000
241069,Ã?Â?stlich der Berge.,3,2.666667


In [17]:
popular_df = popular_df[popular_df['num-rating'] > 250].sort_values('avg-rating', ascending=False).head(50)
popular_df.head()

,Book-Title,num-rating,avg-rating
80434,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804
80422,Harry Potter and the Goblet of Fire (Book 4),387,5.824289
80441,Harry Potter and the Sorcerer's Stone (Book 1),278,5.737410
80426,Harry Potter and the Order of the Phoenix (Boo...,347,5.501441
80414,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453


In [18]:
popular_df = popular_df.merge(books, on='Book-Title').drop_duplicates('Book-Title')[['Book-Title', 'Book-Author', 'Image-URL-M', 'num-rating', 'avg-rating']]

In [19]:
popular_df.columns

Index(['Book-Title', 'Book-Author', 'Image-URL-M', 'num-rating', 'avg-rating'], dtype='object')

In [20]:
x = ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 200
aware_users = x[x].index

In [21]:
filtered_ratings = ratings_with_name[ratings_with_name['User-ID'].isin(aware_users)]

In [22]:
y = filtered_ratings.groupby('Book-Title').count()['Book-Rating'] >= 50
famous_books = y[y].index

In [23]:
final_ratings = filtered_ratings[filtered_ratings['Book-Title'].isin(famous_books)]

In [24]:
final_ratings.drop_duplicates().head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
31,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,11676,9
33,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,36836,0
34,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,46398,9
38,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,113270,0
39,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,113519,0


In [53]:
final_ratings['Image-URL-M']

31         http://images.amazon.com/images/P/0399135782.0...
33         http://images.amazon.com/images/P/0399135782.0...
34         http://images.amazon.com/images/P/0399135782.0...
38         http://images.amazon.com/images/P/0399135782.0...
39         http://images.amazon.com/images/P/0399135782.0...
                                 ...                        
1028414    http://images.amazon.com/images/P/1878702831.0...
1028600    http://images.amazon.com/images/P/0394429869.0...
1028602    http://images.amazon.com/images/P/0449001164.0...
1028815    http://images.amazon.com/images/P/0743527631.0...
1028817    http://images.amazon.com/images/P/0745168086.0...
Name: Image-URL-M, Length: 58586, dtype: object

In [25]:
final_ratings = final_ratings[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-M', 'User-ID', 'Book-Rating']]

In [26]:
final_ratings.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-M', 'User-ID', 'Book-Rating'],
      dtype='object')

In [27]:
round(final_ratings.loc[final_ratings['Book-Title'] == "The Kitchen God's Wife"]['Book-Rating'].mean(), 1)

1.8

In [28]:
pt = final_ratings.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating')

In [29]:
pt.fillna(0, inplace=True)

In [30]:
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
similarity_score = cosine_similarity(pt)

In [33]:
similarity_score.shape

(706, 706)

In [34]:
def recommend(book_name):
    index = np.where(pt.index == book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_score[index])), key=lambda x:x[1], reverse=True)[1:6]
    
    for i in similar_items:
        print(pt.index[i[0]])

In [35]:
recommend('1984')

Animal Farm
The Handmaid's Tale
Brave New World
The Vampire Lestat (Vampire Chronicles, Book II)
The Hours : A Novel


In [36]:
def book_info(book_name):
    index = np.where(final_ratings['Book-Title'] == book_name)[0][0]
#     print(index)
    cover = final_ratings.iloc[index]['Image-URL-M']
    author = final_ratings.iloc[index]['Book-Author']
    publisher = final_ratings.iloc[index]['Publisher']
    year = final_ratings.iloc[index]['Year-Of-Publication']
    print(cover)
    print(author)
    print(publisher)
    print(year)

In [37]:
book_info("Echoes")

http://images.amazon.com/images/P/0440122090.01.MZZZZZZZ.jpg
Maeve Binchy
Dell
1989


In [38]:
final_ratings.iloc[8312]

ISBN                                                          0440122090
Book-Title                                                        Echoes
Book-Author                                                 Maeve Binchy
Year-Of-Publication                                                 1989
Publisher                                                           Dell
Image-URL-M            http://images.amazon.com/images/P/0440122090.0...
User-ID                                                             6563
Book-Rating                                                            0
Name: 37910, dtype: object

In [39]:
import pickle

In [40]:
pickle.dump(final_ratings, open('final_ratings.pkl', 'wb'))
# pickle.dump(pt, open('pt.pkl', 'wb'))
# pickle.dump(similarity_score, open('similarity.pkl', 'wb'))

In [41]:
pip install streamlit-authenticator

Note: you may need to restart the kernel to use updated packages.


In [42]:
import streamlit_authenticator as stauth

user_ids = ["abc", "xyz", "pqr"]
passwords = ["123", "123", "123"]

hashed_passwords = stauth.Hasher(passwords).generate()

In [43]:
# pickle.dump(hashed_passwords, open('passwords.pkl', 'wb'))

In [44]:
df = final_ratings

In [45]:
df.shape

(58586, 8)

In [46]:
df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-M,User-ID,Book-Rating
31,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,11676,9
33,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,36836,0
34,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,46398,9
38,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,113270,0
39,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,113519,0


In [47]:
num_df = df.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_df.rename(columns = {'Book-Rating': 'num-rating'}, inplace=True)
num_df

,Book-Title,num-rating
0,1984,70
1,1st to Die: A Novel,160
2,2nd Chance,122
3,4 Blondes,70
4,A Bend in the Road,114
...,...,...
701,Year of Wonders,57
702,You Belong To Me,55
703,Zen and the Art of Motorcycle Maintenance: An ...,62
704,Zoya,59


In [48]:
avg_df = df.groupby('Book-Title').mean()['Book-Rating'].reset_index()
avg_df.rename(columns = {'Book-Rating': 'avg-rating'}, inplace=True)
avg_df.shape

(706, 2)

In [49]:
p_df = num_df.merge(avg_df, on='Book-Title').drop_duplicates('Book-Title')
p_df

,Book-Title,num-rating,avg-rating
0,1984,70,3.200000
1,1st to Die: A Novel,160,2.375000
2,2nd Chance,122,2.475410
3,4 Blondes,70,0.985714
4,A Bend in the Road,114,1.622807
...,...,...,...
701,Year of Wonders,57,3.070175
702,You Belong To Me,55,1.545455
703,Zen and the Art of Motorcycle Maintenance: An ...,62,1.919355
704,Zoya,59,1.237288


In [50]:
p_df = p_df[p_df['num-rating'] > 200].sort_values('avg-rating', ascending=False).head(50)
p_df.reset_index(inplace=True)
p_df

,index,Book-Title,num-rating,avg-rating
0,570,The Lovely Bones: A Novel,261,3.203065
1,512,The Da Vinci Code,216,3.157407
2,611,The Secret Life of Bees,204,2.848039
3,87,Bridget Jones's Diary,263,2.596958
4,528,The Firm,219,2.150685
5,22,A Time to Kill,208,2.110577
6,579,The Notebook,238,2.029412
7,582,The Pelican Brief,229,1.995633
8,576,The Nanny Diaries: A Novel,221,1.959276
9,17,A Painted House,222,1.954955


In [51]:
p_df['Book-Title']

0                           The Lovely Bones: A Novel
1                                   The Da Vinci Code
2                             The Secret Life of Bees
3                               Bridget Jones's Diary
4                                            The Firm
5                                      A Time to Kill
6                                        The Notebook
7                                   The Pelican Brief
8                          The Nanny Diaries: A Novel
9                                     A Painted House
10    Divine Secrets of the Ya-Ya Sisterhood: A Novel
11                                The Horse Whisperer
12                                Message in a Bottle
13                                        Wild Animus
Name: Book-Title, dtype: object

In [52]:
popular = p_df.merge(final_ratings, on='Book-Title').drop_duplicates('Book-Title').head(10)
popular

,index,Book-Title,num-rating,avg-rating,ISBN,Book-Author,Year-Of-Publication,Publisher,Image-URL-M,User-ID,Book-Rating
0,570,The Lovely Bones: A Novel,261,3.203065,0316666343,Alice Sebold,2002,"Little, Brown",http://images.amazon.com/images/P/0316666343.0...,277427,0
261,512,The Da Vinci Code,216,3.157407,0385504209,Dan Brown,2003,Doubleday,http://images.amazon.com/images/P/0385504209.0...,277427,8
477,611,The Secret Life of Bees,204,2.848039,0142001740,Sue Monk Kidd,2003,Penguin Books,http://images.amazon.com/images/P/0142001740.0...,277427,0
681,87,Bridget Jones's Diary,263,2.596958,0330332775,Helen Fielding,1997,Picador (UK),http://images.amazon.com/images/P/0330332775.0...,11676,0
944,528,The Firm,219,2.150685,0385416342,John Grisham,1991,Doubleday Books,http://images.amazon.com/images/P/0385416342.0...,11676,7
1163,22,A Time to Kill,208,2.110577,0440211727,JOHN GRISHAM,1992,Dell,http://images.amazon.com/images/P/0440211727.0...,277639,8
1371,579,The Notebook,238,2.029412,0446605239,Nicholas Sparks,1998,Warner Books,http://images.amazon.com/images/P/0446605239.0...,278418,0
1609,582,The Pelican Brief,229,1.995633,0440214041,John Grisham,1993,Dell,http://images.amazon.com/images/P/0440214041.0...,277639,9
1838,576,The Nanny Diaries: A Novel,221,1.959276,0312278586,Emma McLaughlin,2002,St. Martin's Press,http://images.amazon.com/images/P/0312278586.0...,3363,0
2059,17,A Painted House,222,1.954955,044023722X,John Grisham,2001,Dell Publishing Company,http://images.amazon.com/images/P/044023722X.0...,278418,0
